In [40]:
import pandas as pd
from collections import Counter
import os
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta
import numpy as np
from scipy import stats
import json

data_dir = 'data/exp1'
min_occurances = 0

df = pd.DataFrame()
for file in os.listdir(data_dir):
    if file.startswith('Session'):
        with open(f'{data_dir}/{file}') as f:
            starttime_millis = int(f.readline().rstrip()[17:])
        session = pd.read_csv(f'{data_dir}/{file}', skiprows=1)
    else:
        tmp = pd.read_csv(f'{data_dir}/{file}', skiprows=1)
        df = pd.concat([df, tmp], ignore_index=True)
df = df.sort_values(by=['timestamp'], ignore_index=True)

session_times = []
for start, end in zip(session['Start Time'], session['Stop Time']):
    delta = [str(timedelta(milliseconds = start - starttime_millis)), 
             str(timedelta(milliseconds = end - starttime_millis))]
    session_times.append(delta)
session[['start formatted', 'end formatted']] = session_times

timestamp_formatted = []
timestamp_start = df['timestamp'][0]
for time in df['timestamp']:
    new_time = str(timedelta(milliseconds=((time-timestamp_start)/1e6)))    #%H:%M:%S:%f
    timestamp_formatted.append(new_time)

timestamp_formatted[0] += ".000000"
df['timestamp_formatted'] = timestamp_formatted

# Get offset to real time
df.to_csv(f'processed-{data_dir.split("/")[1]}.csv')
session.to_csv(f'processed-session-{data_dir.split("/")[1]}.csv')

annotation_json = [{
    "data": {"timeseriesUrl": "/data/upload/8/5261c304-processed-exp1.csv"},
    "predictions": [{"id":3,
                     "completed_by":1,
                     "result":[{"value":{"start":182697357010535,
                                         "end":182704065710535,
                                         "instant":False,
                                         "timeserieslabels":["Other"]},
                                "id":"dxaaW-8yav",
                                "from_name":"label",
                                "to_name":"ts",
                                "type":"timeserieslabels",
                                "origin":"manual"}],
                      "was_cancelled":False,
                      "ground_truth":False,
                      "created_at":"2022-08-23T19:21:52.347617Z",
                      "updated_at":"2022-08-23T19:21:54.319152Z",
                      "lead_time":17.126,
                      "prediction":{},
                      "result_count":0,
                      "task":10,
                      "parent_prediction":None,
                      "parent_annotation":None}]

}]
with open(f'annotations-{data_dir.split("/")[1]}.json', 'w+') as f:
    json.dump(annotation_json,f)